In [7]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
from torchvision import datasets, models
import matplotlib.pyplot as plt
from data_utils import FaceData, FaceDataCropped
from nets.convage_final import ConvAgeNN
import time
import os

%load_ext autoreload
%autoreload 2

plt.ion()   # interactive mode

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
train_data = FaceDataCropped(image_paths_file='LAG_background/train/train.txt')
val_data = FaceData(image_paths_file='LAG/val/val.txt')
test_data = FaceData(image_paths_file='LAG/train/train.txt')

In [9]:
model = torch.load('models/convage_segnet_nobackground/convage_segnet_nobackground_760_epochs.model')
model.cpu()
num_example_imgs = 5
index = [25, 40, 44, 47, 68, 92]
numRange = np.floor(len(val_data) / num_example_imgs) 
for i, (img, target) in enumerate(val_data[index]):
    plt.figure(figsize=(15, 5))
    inputs = img.unsqueeze(0)
    inputs = Variable(inputs)
    if model.is_cuda:
        inputs = inputs.cuda()
    outputs = model.forward(inputs)
    _, preds = torch.max(outputs, 1)
    pred = preds[0].data.cpu()

    img, target, pred = img.numpy(), target.numpy(), pred.numpy()

    # img
    plt.subplot(1, 3, 1)
    plt.axis('off')
    plt.imshow(img.transpose(1,2,0))
    plt.title("Input image")

    # target
    plt.subplot(1, 3, 2)
    plt.axis('off')
    target /= 255
    plt.imshow(target.reshape(200,200), cmap='gray')
    plt.title("Target image")

    # pred
    plt.subplot(1, 3, 3)
    plt.axis('off')
    plt.imshow(pred.reshape(200,200), cmap='gray')
    plt.title("Prediction image")
    plt.savefig('results/convage_nn_background/650_epochs_' + str(i) + '.png')
    plt.show()


AttributeError: Can't get attribute 'SegmentationNN' on <module 'classifiers.segmentation_nn' from '/home/carlodedonno/Documents/Code/ConvAge/classifiers/segmentation_nn.py'>

In [ ]:
for i in range(5):
    startEpoch = 120
    intEpochs = 20
    model = torch.load('models/convage_mse_nobackground_' + str(startEpoch + i * intEpochs) + '_epochs.model')
    model.cpu()
    
    num_example_imgs = 3
    plt.figure(figsize=(15, 5 * num_example_imgs))
    
    for i, (img, target) in enumerate(val_data[-num_example_imgs:]):
        inputs = img.unsqueeze(0)
        inputs = Variable(inputs)
        if model.is_cuda:
            inputs = inputs.cuda()

        outputs = model.forward(inputs)
        _, preds = torch.max(outputs, 1)
        pred = outputs[0].data.cpu()

        img, target, pred = img.numpy(), target.numpy(), pred.numpy()

        # img
        plt.subplot(num_example_imgs, 3, i * 3 + 1)
        plt.axis('off')
        plt.imshow(img.transpose(1,2,0))
        if i == 0:
            plt.title("Input image")

        # target
        plt.subplot(num_example_imgs, 3, i * 3 + 2)
        plt.axis('off')
        target /= 255
        plt.imshow(target.reshape(200,200), cmap='gray')
        if i == 0:
            plt.title("Target image")

        # pred
        plt.subplot(num_example_imgs, 3, i * 3 + 3)
        plt.axis('off')
        plt.imshow(pred.reshape(200,200), cmap='gray')
        if i == 0:
            plt.title("Prediction image")
    plt.savefig('results/convage_mse_nobackground_' + str(startEpoch + i * intEpochs) + '_epochs.png')
    plt.show()